In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle

In [2]:
con = sqlite3.connect("sqlite:///../Models/2019.sqlite")
cur = con.cursor()

In [4]:
C = pd.read_csv("Resources/19C.csv")
first = pd.read_csv("Resources/191B.csv")
second = pd.read_csv("Resources/192B.csv")
short = pd.read_csv("Resources/19SS.csv")
third = pd.read_csv("Resources/193B.csv")
OF = pd.read_csv("Resources/19OF.csv")

In [5]:
total = C
total = total.append(first)
total = total.append(second)
total = total.append(short)
total = total.append(third)
total = total.append(OF)

In [57]:
leveler = {'A+':3, 'A': 4, 'R':6, 'A-':5, 'AAA': 1, 'AA':2, 'MLB': 0}
releveler = {3:'A+', 4:'A', 6:'Rk', 5:'A-', 1: 'AAA', 2 : 'AA', 0: 'MLB'}

In [7]:
total.to_csv('Resources/total19.csv')

In [8]:
total = pd.read_csv('Resources/total19.csv')

In [10]:
positAdjuster = {'P' : 0, 'SS': 7.5, '2B': 2.5, 'OF': -7.5, '3B-2B': 2.5, '2B-3B': 2.5, 'SS-2B': 7.5, '3B': 2.5, 'OF-1B': -7.5,
'C': 12.5, 'OF-2B': 2.5, '2B-OF': 2.5, '1B': -12.5, 'UTL': 0, 'C-1B': 12.5, '3B-C': 12.5, 'C-3B': 12.5,
'3B-SS': 7.5, 'IF': 2.5, 'OF-3B': 2.5, 'OF-P': -7.5, 'SS-3B': 7.5, '1B-OF': -7.5, '3B-OF': 2.5, '1B-P': -12.5,
'P-C': 12.5, 'P-OF': -7.5, '2B-SS': 7.5, '2B-P': 2.5, '3B-1B': 2.5, 'P-1B': -12.5, 'OF-C': 12.5, '1B-3B': 2.5,
'P-3B': 2.5, 'C-OF': 12.5, '1B-C': 12.5, 'P-IF': 2.5, 'OF-SS': 7.5, '3B-P': 2.5, 'SS-OF': 7.5, '2B-C': 12.5,
'SS-P': 7.5, 'H': 0, 'LF': -7.5, 'P-2B': 2.5, 'C-2B': 12.5, 'C-P': 12.5, 'RF': -7.5, 'CF': 2.5, 'RF-LF': -7.5,
'LF-CF': 2.5, 'LF-RF': -7.5, 'LF-2B': 2.5, '2B-LF': 2.5, 'CF-RF': 2.5, 'OF-DH': -7.5, '1B-LF': -7.5,
'CF-LF': 2.5, 'LF-1B': -7.5, '1B-DH': -12.5, 'LF-OF': -7.5, '1B-2B': 2.5, 'DH': -17.5, 'C-DH': 12.5, 'RF-CF': -7.5,
'RF-OF': -7.5, 'DH-OF': -7.5, 'DH-CF': 2.5, '2B-CF': 2.5, 'RF-1B': -7.5, 'DH-1B': -12.5, 'SS-CF': 7.5,
'CF-OF': 2.5, '1B-RF': -7.5, '1B-SS': 7.5, 'DH-3B': 2.5, 'C-IF': 12.5, '3B-LF': 2.5, 'CF-3B': 2.5,
'2B-1B': 2.5, 'RF-3B': 2.5, 'IF-P': 2.5, 'CF-2B': 2.5, '3B-RF': 2.5, 'SS-1B': 7.5, 'IF-C': 12.5,
'SS-C': 12.5, 'LF-C': 12.5, '3B-DH': 2.5, 'SS-LF': 7.5, 'INF': 2.5, 'RF-2B': 2.5, 'C-LF': 12.5, 'P-SS': 7.5,
'CF-1B': 2.5, 'C-SS': 12.5, 'LF-SS': 7.5, 'C-RF': 12.5, '3B-CF': 2.5, 'P-DH': -17.5, '2B-RF': 2.5,
'LF-3B': 2.5, 'CF-SS': 7.5, 'SS-RF': 7.5, 'DH-LF': -7.5, 'RF-P': -7.5, 'DH-C': 12.5, 'P-LF': -7.5, 'UT-P': 0,
'LF-DH': -7.5, 'RF-C': 12.5}

In [13]:
total['Def'] = total.Position
total = total.replace({"Def" : positAdjuster})

In [15]:
data = total

In [16]:
data['BABIP'] = (data.H - data.HR) / (data.AB - data.SO - data.HR + data.SF)
data['SB%'] = data.SB / (data.SB + data.CS)
data['SB%'] = data['SB%'].fillna(0)
data['SLG'] = (data['1B'] + (data['2B'] *2) + (data['3B'] * 3) + (data.HR * 4))/ data.AB
data['BB%'] = (data.BB + data.IBB) / data.PA
data['K%'] = data.SO / data.PA
data['ISO'] = data.SLG - data.AVG

In [28]:
look = data.groupby(['Level']).transform(lambda x: (x - x.mean()) / x.std())

C:\Users\alana\Anaconda3\lib\site-packages\pandas\core\ops.py:1037: RuntimeWarning: invalid value encountered in double_scalars
  lambda x: op(x, rvalues))


In [30]:
look.columns

Index(['Season', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG',
       'Def', 'BABIP', 'SB%', 'SLG', 'BB%', 'K%', 'ISO'],
      dtype='object')

In [32]:
data.columns

Index(['Season', 'Name', 'Team', 'Level', 'Age', 'G', 'AB', 'PA', 'H', '1B',
       '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH',
       'GDP', 'SB', 'CS', 'AVG', 'PlayerId', 'Position', 'Def', 'BABIP', 'SB%',
       'SLG', 'BB%', 'K%', 'ISO'],
      dtype='object')

In [33]:
final = data[['Name', 'Team', 'Level', 'PA', 'PlayerId', 'Position']]

In [35]:
final[['AVG', 'SLG', 'Age', 'DEF', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP']] = look[['AVG', 'SLG', 'Age',  'Def', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP']]

C:\Users\alana\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [58]:
final = final.replace({"Level" : leveler})

In [44]:
filename = "Models/randomforest.h5"

In [45]:
rf = pickle.load(open(filename, 'rb'))

In [59]:
X = final[['Level', 'Age', 'DEF', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP']]

In [61]:
final['Prediction'] = rf.predict(X)

In [82]:
final[final.Name == 'Willians Astudillo']

,Name,Team,Level,PA,PlayerId,Position,AVG,SLG,Age,DEF,ISO,BB%,K%,SB%,BABIP,Prediction
29,Willians Astudillo,Twins,1,40,15608,C,4.884215,2.68124,0.07718,1.586740,0.594983,-1.781367,-2.771388,-0.008325,2.689669,0
3477,Willians Astudillo,Twins,1,40,15608,OF,4.884215,2.68124,0.07718,-0.834519,0.594983,-1.781367,-2.771388,-0.008325,2.689669,1


In [83]:
final = final.replace({"Level" : releveler, "Prediction" : releveler})

In [87]:
final.to_sql('predictiondata', con, if_exists='replace', index=False)

In [5]:
example = pd.read_sql_query("SELECT * from predictiondata", con)

In [6]:
final = final.round({'AVG': 3, 'SLG': 3, 'OBP': 3, 'OPS': 3, 'Age': 3, 'DEF': 3, 'ISO': 3, 'BB%': 3, 'K%': 3, 'SB%': 3, 'BABIP': 3, 'DaysOld': 3})
final.head()

,Name,Team,Level,PA,PlayerId,Position,AVG,SLG,Age,DEF,ISO,BB,K,SB,BABIP,Prediction
0,Tomas Telis,Twins,AAA,241,10067,C,0.711422,0.151316,0.391896,1.58674,-0.241976,-1.284124,-1.583025,-1.318801,0.047754,MLB
1,Francisco Arcia,Cubs,AAA,171,10286,C,-1.764042,-2.202081,0.706612,1.58674,-1.909688,-1.397516,-0.782137,-1.318801,-1.752742,AAA
2,John Ryan Murphy,Diamondbacks,AAA,136,10346,C,-0.439461,0.562066,0.391896,1.58674,1.050025,-0.081068,0.454594,-1.318801,-0.863828,MLB
3,Rocky Gale,Dodgers,AAA,109,10533,C,-0.439461,-0.805346,1.336045,1.58674,-0.828472,-0.836649,0.027094,1.302151,-0.281921,MLB
4,David Freitas,Brewers,AAA,269,10619,C,2.071789,0.726495,1.021329,1.58674,-0.312008,0.392334,-0.944531,-1.318801,1.867404,MLB
